In [56]:
# Autor: Jairo Valea López
#
# Rosin-Rammler

# Importado de librerías habituales

import os
import math
import pandas as pd
import numpy as np
import scipy.stats as stats
import time
import matplotlib
import natsort
import matplotlib.pylab as pl
import seaborn as sns
import matplotlib.patches as mpatches
from tqdm.notebook import tqdm, trange # barra de progreso
from matplotlib import pyplot as plt
from matplotlib import gridspec
from natsort import natsorted
from matplotlib.colors import ListedColormap

# Funciones propias

def promedio(lst):
    return sum(lst) / len(lst)

def abline(slope, intercept):
    """Plot a line from slope and intercept"""
    x_vals = np.array(ax.get_xlim())
    y_vals = intercept + slope * x_vals
    ax.plot(x_vals, y_vals, '--', color = 'red')
    
def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return array[idx]

def acumulador(m):
    granos = np.array(m.iloc[0:m.shape[0],45:76])
    densidad = np.empty((granos.shape[0],granos.shape[1]))
    masas = np.empty((granos.shape[0],granos.shape[1]))
    masas_ac = np.empty((granos.shape[0],granos.shape[1]))
    for k in range(granos.shape[0]):
            for l in range(granos.shape[1]):
                densidad[k,l] = (granos[k,l])*(1/dx[l+41])
                masas[k,l] = (round((4/3)*densidad[k,l]*(3.141592/8)*(diams_g[l])**3,4)) # masa
                masas_ac[k] = np.cumsum(masas[k,:])
    for i in range(masas_ac.shape[0]):
            masas_ac[i] = (masas_ac[i])/max(masas_ac[i])
    return masas_ac

def rosinrammler(ac, m):
    x = np.linspace (0, 20, 100)
    cerca = find_nearest(ac, 0.632)
    res = np.where(ac == cerca)[0][0]
    if (cerca > 0.632):
        while (ac[res] == ac[res-1]):
            res = res-1        
        diam_tipo = diams_g[res]-((diams_g[res] - diams_g[res-1])*(ac[res] - 0.632) / (ac[res] - ac[res-1]))
    if (cerca < 0.632):
        while (ac[res] == ac[res+1]):
            res = res+1
        diam_tipo = diams_g[res+1]-((diams_g[res+1] - diams_g[res])*(ac[res+1] - 0.632) / (ac[res+1] - ac[res]))
    if (cerca == 0.632):
        diam_tipo = diams_g[res]
    F = 1 - 2.71828**(-(x/diam_tipo)**m)
    return F

In [82]:
#
# Procesado para separar por visibilidades
#

# CAMBIAR OPCIONES

opciones = ['JunJul/', 'AgoSep/']

ww = 0

ruta_proces = 'C:/Users/miguel.anton/Desktop/NIEBLA/Fractal/Datos/Rosin-Rammler/' + opciones[ww]  # cambiar ruta

carpeta = natsorted(os.listdir(ruta_proces))
procesados = []
nombres = []
niebla = []

for f in carpeta:
    name, ext = os.path.splitext(f)
    if ext == '.txt':
        procesados.append(pd.read_csv(ruta_proces + name + ext, delimiter = ",", decimal = "."))
        nombres.append(name + ext)
        
diams_g = [2.13,2.289,2.46,2.643,2.841,3.053,3.28,3.525,3.788,4.071,4.374,4.701,5.051,5.428,5.833,6.268,6.736,7.239,
           7.779,8.359,8.983,9.653,10.373,11.147,11.979,12.872,13.833,14.865,15.974,17.165,18.446]
diams = [0.104,0.111,0.12,0.129,0.138,0.149,0.16,0.172,0.184,0.198,0.213,0.229,0.246,0.264,0.284,0.305,0.328,0.352,0.379,
         0.407,0.437,0.47,0.505,0.543,0.583,0.627,0.674,0.724,0.778,0.836,0.898,0.965,1.037,1.115,1.198,1.287,1.383,1.486,
         1.597,1.717,1.845,1.982,2.13,2.289,2.46,2.643,2.841,3.053,3.28,3.525,3.788,4.071,4.374,4.701,5.051,5.428,5.833,
         6.268,6.736,7.239,7.779,8.359,8.983,9.653,10.373,11.147,11.979,12.872,13.833,14.865,15.974,17.165,18.446]

dx = [0.007,0.008,0.009,0.009,0.01,0.011,0.011,0.012,0.013,0.014,0.015,0.016,0.018,0.019,0.02,0.022,0.024,0.025,0.027
,0.029,0.031,0.034,0.036,0.039,0.042,0.045,0.048,0.052,0.056,0.06,0.065,0.069,0.075,0.08,0.086,0.093,0.099,0.107,0.115
,0.123,0.133,0.143,0.153,0.165,0.177,0.19,0.204,0.22,0.236,0.254,0.272,0.293,0.315,0.338,0.363,0.39,0.42,0.451,0.484
,0.521,0.559,0.601,0.646,0.694,0.746,0.802,0.862,0.926,0.995,1.069,1.149,1.235,1.327]

for i in range(len(nombres)):
    if (nombres[i][6] == '8'):
        procesados[i] = procesados[i].apply(lambda col:pd.to_numeric(col, errors='coerce'))
        procesados[i] = procesados[i].dropna()
        procesados[i] = procesados[i][procesados[i]['Visibilidad corregida (m)'] != 0]
        procesados[i].reset_index(drop=True, inplace=True)
        procesados[i]['Ensayo'] = '8.' + nombres[i][8:10]
        procesados[i]['Visibilidad corregida (m)'] = procesados[i]['Visibilidad corregida (m)'].mask(procesados[i]['Visibilidad corregida (m)'] > 2000, 2000)
        #
        # "LIMPIADOR" DE LOS DATOS CON DIFERENCIA DE VISIBILIDAD > X - ANTES DE APPEND
        #
        rolling_mean = procesados[i].iloc[:,77].rolling(5, min_periods = 1).mean()
        if (len(procesados[i]) > 0):
            for j in range(len(procesados[i]) - 1):
                if (abs(rolling_mean[j+1] - rolling_mean[j]) > 5):
                    procesados[i] = procesados[i].drop(j+1)
        procesados[i].reset_index(drop=True, inplace=True)
        procesados[i]['Ensayo'] = '8.' + nombres[i][8:10]
        #
        # FIN DEL LIMPIADOR DE DATOS
        #
        niebla.append(procesados[i])

total = pd.concat(niebla,ignore_index = True)

In [83]:
# Operaciones primarias

Fd = []
lnd = []
lnd = np.log(diams_g)

ac = acumulador(total)
Fd = (np.log(-np.log(1.0001-ac)))
    
# Rampa de colores, por visibilidad (diferenciar ensayos)

color_labels = natsorted(total['Ensayo'].unique())
rgb_values = sns.color_palette("hls", len(color_labels))
colorinchos = ListedColormap(sns.color_palette(rgb_values).as_hex())
color_map = dict(zip(color_labels, rgb_values))


l_v = [10,20,30,40,50,60,70,80,100,120,150,500,2001] # límites rangos de visibilidad

<ipython-input-83-a66388b6d582>:8: RuntimeWarning: invalid value encountered in log
  Fd = (np.log(-np.log(1.0001-ac)))


In [84]:
# Rosin-Rammler por rangos de visibilidad

ruta_save = 'C:/Users/miguel.anton/Desktop/NIEBLA/Fractal/Datos/Rosin-Rammler/Gráficos/'
x = np.linspace (0, 20, 100)
hand = []

for i in range(len(rgb_values)):
    hand.append(mpatches.Patch(color=rgb_values[i], label=color_labels[i]))

for z in range(len(l_v)-1):
    ac_vis = ac[(total['Visibilidad corregida (m)'] > l_v[z]) & (total['Visibilidad corregida (m)'] < l_v[z+1])]
    Fd_vis = Fd[(total['Visibilidad corregida (m)'] > l_v[z]) & (total['Visibilidad corregida (m)'] < l_v[z+1])]
    datos_vis = total[(total['Visibilidad corregida (m)'] > l_v[z]) & (total['Visibilidad corregida (m)'] < l_v[z+1])]
    if (not datos_vis.empty):
        slope = []
        intercept = []
        lines = []
        ros = []
        fig = plt.figure(figsize=(12,7))
        gs = gridspec.GridSpec(1, 2, width_ratios = [1, 1])
        ax = plt.subplot(gs[0,0])
        ax3 = plt.subplot(gs[0,1])
        ax.set_title('Visibilidad de ' + str(l_v[z]) + ' a ' + str(l_v[z+1]) + ' m, ensayos de ' + opciones[ww][0:-1])
        ax.set_xlabel('ln d')
        ax.set_ylabel('ln(-ln(1-Fd)')
        ax.set_xlim(0.5,3)
        ax.set_ylim(-8,4)
        for i in (range(len(Fd_vis))):
            line,= ax.plot(lnd[Fd_vis[i] < 2.2203],Fd_vis[i][Fd_vis[i] < 2.2203], c = datos_vis['Ensayo'].map(color_map)[datos_vis.index[i]], alpha = 0.5)
            lines.append(line)
            if (len(Fd_vis[i][4:-10][Fd_vis[i][4:-10] < 2.2203]) > 0):
                pend, orden,_,_,_ = stats.linregress(lnd[4:-10][Fd_vis[i][4:-10] < 2.2203],Fd_vis[i][4:-10][Fd_vis[i][4:-10] < 2.2203])
                slope.append(pend)
                intercept.append(orden)
        if slope:
            abline(promedio(slope),promedio(intercept))
            ax.annotate(text = 'Pendiente: ' + str(round(promedio(slope),3)), xy = (2.0,-7.6), ha = 'right', c = 'red')
            ros = rosinrammler(promedio(ac_vis),promedio(slope))
        ax.axvline(x = lnd[4], ls = 'dotted', linewidth = 0.5, color = 'black')
        ax.axvline(x = lnd[-10], ls = 'dotted', linewidth = 0.5, color = 'black')
        ax3.scatter(diams_g, promedio(ac_vis), marker = '.')
        ax3.plot(x, ros, color = 'red')
        #ax3.yaxis.tick_right()
        ax3.set_xlabel('Diámetro (um)')
        ax3.set_ylabel('Distribución')
        ax3.set_xlim([0,20])
        ax3.set_ylim([0,1])
        ax3.set_yticks([0,0.25,0.5,0.75,1])
        ax3.set_yticklabels(['0%','25%','50%','75%','100%'])
        ax3.grid(True)
        ax.legend(handles=hand, loc = 'lower right')
        plt.tight_layout()
        plt.savefig(ruta_save + opciones[ww][0:-1] + '_rosin_' + str(l_v[z]) + ' a ' + str(l_v[z+1]) + '.png')
        plt.close(fig)

C:\Users\miguel.anton\Anaconda3\lib\site-packages\scipy\stats\_stats_mstats_common.py:160: RuntimeWarning: invalid value encountered in double_scalars
  slope = ssxym / ssxm
C:\Users\miguel.anton\Anaconda3\lib\site-packages\scipy\stats\_stats_mstats_common.py:174: RuntimeWarning: invalid value encountered in sqrt
  t = r * np.sqrt(df / ((1.0 - r + TINY)*(1.0 + r + TINY)))
C:\Users\miguel.anton\Anaconda3\lib\site-packages\scipy\stats\_stats_mstats_common.py:176: RuntimeWarning: invalid value encountered in double_scalars
  slope_stderr = np.sqrt((1 - r**2) * ssym / ssxm / df)


In [ ]:
#
# Procesado para separar por ensayos
#

ruta_proces = 'C:/Users/miguel.anton/Desktop/NIEBLA/Fractal/Datos/'  # cambiar ruta

carpeta = natsorted(os.listdir(ruta_proces))
procesados = []
nombres = []
niebla = []

for f in carpeta:
    name, ext = os.path.splitext(f)
    if ext == '.txt':
        procesados.append(pd.read_csv(ruta_proces + name + ext, delimiter = ",", decimal = "."))
        nombres.append(name + ext)
        
diams_g = procesados[0].iloc[60,45:76]
diams = procesados[0].iloc[60,3:76]

dx = [0.007,0.008,0.009,0.009,0.01,0.011,0.011,0.012,0.013,0.014,0.015,0.016,0.018,0.019,0.02,0.022,0.024,0.025,0.027
,0.029,0.031,0.034,0.036,0.039,0.042,0.045,0.048,0.052,0.056,0.06,0.065,0.069,0.075,0.08,0.086,0.093,0.099,0.107,0.115
,0.123,0.133,0.143,0.153,0.165,0.177,0.19,0.204,0.22,0.236,0.254,0.272,0.293,0.315,0.338,0.363,0.39,0.42,0.451,0.484
,0.521,0.559,0.601,0.646,0.694,0.746,0.802,0.862,0.926,0.995,1.069,1.149,1.235,1.327]

for i in range(len(nombres)):
    if (nombres[i][6] == '8'):
        procesados[i] = procesados[i].apply(lambda col:pd.to_numeric(col, errors='coerce'))
        #procesados[i] = procesados[i].dropna()
        procesados[i] = procesados[i][procesados[i]['Visibilidad corregida (m)'] != 0]
        procesados[i].reset_index(drop=True, inplace=True)
        procesados[i]['Ensayo'] = '8.' + nombres[i][8:10]
        procesados[i]['Visibilidad corregida (m)'] = procesados[i]['Visibilidad corregida (m)'].mask(procesados[i]['Visibilidad corregida (m)'] > 2000, 2000)
        #
        # "LIMPIADOR" DE LOS DATOS CON DIFERENCIA DE VISIBILIDAD > X - ANTES DE APPEND
        #
        rolling_mean = procesados[i].iloc[:,77].rolling(5, min_periods = 1).mean()
        if (len(procesados[i]) > 0):
            for j in range(len(procesados[i]) - 1):
                if (abs(rolling_mean[j+1] - rolling_mean[j]) > 5):
                    procesados[i] = procesados[i].drop(j+1)
        procesados[i].reset_index(drop=True, inplace=True)
        procesados[i]['Ensayo'] = '8.' + nombres[i][8:10]
        #
        # FIN DEL LIMPIADOR DE DATOS
        #
        niebla.append(procesados[i])

total = pd.concat(niebla,ignore_index = True)

In [302]:
# Rampa de colores, por ensayo (diferenciar visibilidades)

Fd = []
lnd = []
lnd = np.log(diams_g)

for i in range(len(niebla)):
    m = niebla[i]
    ac = acumulador(m)
    Fd.append(np.log(-np.log(1.0001-ac)))
    
color_labels = natsorted(total['Visibilidad corregida (m)'].unique())
rgb_values = sns.color_palette("PuBu_d", len(color_labels))
rgb_values[0] = (0.2,1,0.2)
colorinchos = ListedColormap(sns.color_palette(rgb_values).as_hex())
color_map = dict(zip(color_labels, rgb_values))

<ipython-input-302-1cbfb950f0a2>:10: RuntimeWarning: invalid value encountered in log
  Fd.append(np.log(-np.log(1.0001-ac)))


In [307]:
# Por ensayo

ruta_save = 'C:/Users/miguel.anton/Desktop/NIEBLA/Fractal/Datos/Rosin-Rammler/Gráficos/'
x = np.linspace (0, 20, 100)

for z in range(len(Fd)):
    slope = []
    intercept = []
    lines = []
    ros = []
    fig = plt.figure(figsize=(14,7))
    gs = gridspec.GridSpec(1, 3, width_ratios = [15, 1, 15])
    ax = plt.subplot(gs[0,0])
    ax2 = plt.subplot(gs[0,1])
    ax3 = plt.subplot(gs[0,2])
    ax.set_title(str(nombres[z][:-14]))
    ax.set_xlabel('ln d')
    ax.set_ylabel('ln(-ln(1-Fd)')
    ax.set_xlim(0.5,3)
    ax.set_ylim(-8,4)
    for i in (niebla[z].index):
        line,= ax.plot(lnd[Fd[z][i] < 2.2203],Fd[z][i][Fd[z][i] < 2.2203], c = niebla[z]['Visibilidad corregida (m)'].map(color_map)[i], alpha = 0.3)
        lines.append(line)
        if (len(Fd[z][i][4:-10][Fd[z][i][4:-10] < 2.2203]) > 0):
            pend, orden,_,_,_ = stats.linregress(lnd[4:-10][Fd[z][i][4:-10] < 2.2203],Fd[z][i][4:-10][Fd[z][i][4:-10] < 2.2203])
            slope.append(pend)
            intercept.append(orden)
    if slope:
        abline(promedio(slope),promedio(intercept))
        ax.annotate(text = 'Pendiente: ' + str(round(promedio(slope),3)), xy = (2.9,-7.6), ha = 'right', c = 'red')
        ros = rosinrammler(ac[z],promedio(slope))
    cb = matplotlib.colorbar.ColorbarBase(ax = ax2, cmap = colorinchos, ticks=[0.007, 0.4255, 1])
    cb.ax.set_yticklabels(['15','100','>250'])
    ax2.yaxis.tick_left()
    ax2.set_ylabel('Visibilidad (m)')
    ax.axvline(x = lnd[4], ls = 'dotted', linewidth = 0.5, color = 'black')
    ax.axvline(x = lnd[-10], ls = 'dotted', linewidth = 0.5, color = 'black')
    ax3.scatter(diams_g, ac[z], marker = '.')
    ax3.plot(x, ros, color = 'red')
    #ax3.yaxis.tick_right()
    ax3.set_xlabel('Diámetro (um)')
    ax3.set_ylabel('Distribución')
    ax3.set_xlim([0,20])
    ax3.set_yticks([0,0.25,0.5,0.75,1])
    ax3.set_yticklabels(['0%','25%','50%','75%','100%'])
    ax3.grid(True)
    plt.tight_layout()
    plt.savefig(ruta_save + 'rosin_' + str(nombres[z][:-14]) + '.png')
    plt.close(fig)

C:\Users\miguel.anton\Anaconda3\lib\site-packages\scipy\stats\_stats_mstats_common.py:160: RuntimeWarning: invalid value encountered in double_scalars
  slope = ssxym / ssxm
C:\Users\miguel.anton\Anaconda3\lib\site-packages\scipy\stats\_stats_mstats_common.py:174: RuntimeWarning: invalid value encountered in sqrt
  t = r * np.sqrt(df / ((1.0 - r + TINY)*(1.0 + r + TINY)))
C:\Users\miguel.anton\Anaconda3\lib\site-packages\scipy\stats\_stats_mstats_common.py:176: RuntimeWarning: invalid value encountered in double_scalars
  slope_stderr = np.sqrt((1 - r**2) * ssym / ssxm / df)
